### ingest races.csv file

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Read the file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType


In [0]:
races_schema =  StructType(fields = [StructField("raceId", IntegerType(), False),
                                      StructField("year", IntegerType(), True),
                                      StructField("round", StringType(), True),
                                      StructField("circuitId", IntegerType(), True),
                                      StructField("name", StringType(), True),
                                      StructField("date", DateType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("url", StringType(), True),])

In [0]:
races_df = spark.read \
    .option("header", True) \
    .schema(races_schema) \
    .csv(f"{raw_folder_path}/{v_file_date}/races.csv")

##### Add ingestion date and racetimestamp to the dataframe

In [0]:
from pyspark.sql.functions import col, lit, isnull, replace, concat, coalesce, current_timestamp, to_timestamp

In [0]:
races_none_times_replaced_df = races_df.na.replace('\\N', '00:00:00', subset=["time"])

In [0]:
races_with_timestamp_df = races_none_times_replaced_df.withColumn("race_timestamp", to_timestamp(concat(col('date'), lit 
    (' '), col('time')), 'yyyy-MM-dd HH:mm:ss')) \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date))

In [0]:
races_with_ingestion_date_df = add_ingestion_date(races_with_timestamp_df)

##### Select only the columns required

In [0]:
races_final_df = races_with_ingestion_date_df.select(col('raceId').alias('race_id'), col('year').alias('race_year'), 
                                                   col('round'), col('circuitId').alias('circuit_id'),col('name'), col('ingestion_date'), col('race_timestamp'))


##### Write the outpiut to processed container in parquet format

In [0]:
races_final_df.write.mode("overwrite").partitionBy('race_year').format("delta").saveAsTable("f1_processed.races")

In [0]:
dbutils.notebook.exit("Success")